## Packages

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import  train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix, classification_report

# Benchmark Model
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import BigramTagger
nltk.download("averaged_perceptron_tagger")

# Home-made functions
from functions import *


import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Olivier\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
c:\Users\Olivier\Documents\GitHub\Kazakh-and-multilingual-post-tagging\functions.py:88: SyntaxWarning: invalid escape sequence '\#'
  """


## Import & clean data

In [2]:
path_kz = 'data/kdt-NLANU-0.01.connlu.txt'
path_en = 'data/en_ewt-ud-dev.conllu'
#path_tu = 'data/tr_kenet-ud-dev.conllu'

path_data = [
    path_kz, 
    path_en, 
#    path_tu
    ]

languages = [
    'kazakh',
    'english',
#    'turkish'
    ]

In [3]:
columns = ["ID", "WORD", "LEMMA", "POS", "XPOS", "MORPH", "HEAD", "DEPREL", "DEPS", "MISC"]

for path, lang in tqdm(zip(path_data, languages)) :

    print("____________________________" , lang.upper(), "CORPUS ____________________________")


    # Read the file and convert it to a DataFrame
    df = pd.read_csv(path,
                    sep='\t',
                    names=columns,
                    skip_blank_lines=True
                    )

    # run the hand-made function to clean data
    df = clean_data(df)

    tagged_sentences = data_to_nltk(df)

    X_lex, Y_lex = get_values(df)

    # Split data into training and testing sets
    train_data, test_data = train_test_split(tagged_sentences, 
                                            test_size=0.1, 
                                            random_state=42
                                            )


    ################## NLTK MODEL ##################
    # setup and train BigramTagger
    DFTagger = DefaultTagger("NN")
    Tagger = BigramTagger(train_data, 
                        backoff=DFTagger)

    # Extract true labels and predicted labels from the test data
    Y_test_nltk, predicts_test_nltk = extract_tags(test_data, Tagger)
    Y_train_nltk, predicts_train_nltk = extract_tags(train_data, Tagger)
    ################################################





    X_test, y_test = extract_words_and_tags(test_data)
    X_train, y_train = extract_words_and_tags(train_data)

    #get max word length
    max_word_len = max(max([len(w) for w in Y_lex]), max([len(w) for w in X_lex]))

    #Char2vec model
    vectorizer = TfidfVectorizer(lowercase=False, 
                                analyzer='char'
                                )

    X = vectorizer.fit_transform(X_lex)
    dic = vectorizer.get_feature_names_out() # letter dictionary
    num_letters = len(dic)
    mx = X.T.dot(X) # letter cooccurence matrix
    mx = mx.toarray()

    #Vectorize X only
    X_lex_vec_train = [alpha_vec2(w, mx, max_word_len, dic) for w in X_train]
    X_lex_vec_test = [alpha_vec2(w, mx, max_word_len, dic) for w in X_test]

    # Encode Y
    list_tags = list_all_POS_tags(y = y_train)
    encoder_tag = LabelEncoder().fit(list_tags)

    Y_train = encoder_tag.transform(y_train)
    Y_test = encoder_tag.transform(y_test)

    # Build & train model
    best_model = ExtraTreesClassifier(n_estimators=10,
                                    n_jobs=-1,
                                    criterion='entropy',
                                    bootstrap=True
                                    )

    best_model.fit(X_lex_vec_train, Y_train)

    # predict both train and test sets
    predicts_test = best_model.predict(X_lex_vec_test)
    predicts_train = best_model.predict(X_lex_vec_train)

    
    #####################################################################################################
    ########################################## result analysis ##########################################
    #####################################################################################################
    test_acc, test_f1, test_recall, train_acc, train_f1, train_recall = calculate_results(Y_test, 
                                                                                            Y_train, 
                                                                                            predicts_test, 
                                                                                            predicts_train
                                                                                            )
    test_acc_nltk, test_f1_nltk, test_recall_nltk, train_acc_nltk, train_f1_nltk, train_recall_nltk = calculate_results(Y_test_nltk, 
                                                                                            Y_train_nltk, 
                                                                                            predicts_test_nltk, 
                                                                                            predicts_train_nltk
                                                                                            )

    data = {
        "Metric": ["Test Accuracy", "Test F1 Score", "Test Recall", 
                "Train Accuracy", "Train F1 Score", "Train Recall"],
        "Multi-language POS Tagger": [test_acc, test_f1, test_recall,
                        train_acc, train_f1, train_recall],
        "NLTK POS Tagger": [test_acc_nltk, test_f1_nltk, test_recall_nltk,
                    train_acc_nltk, train_f1_nltk, train_recall_nltk]
    }

    # Create the DataFrame
    df_results = pd.DataFrame(data)

    # Display the DataFrame
    display(df_results)

    
    fig = plot_confusion_matrix(Y_test, predicts_test, list_tags, 'Test set', lang)
    save_graph_to_folder(fig, lang, 'confusion_matrix_test')
    
    fig = plot_confusion_matrix(Y_train, predicts_train, list_tags, 'Train set', lang)
    save_graph_to_folder(fig, lang, 'confusion_matrix_train')

    df_tag_acc = per_tag_accuracy(Y_test, 
                              predicts_test, 
                              list_tags, 
                              encoder_tag
                              )

    display(df_tag_acc) # display accuracy per Tag
    
    df_tag_dist = tag_prediction_nb(
        Y_test, 
        predicts_test, 
        list_tags, 
        encoder_tag
        )

    display(df_tag_dist) # display the number of correct and incorect predictions for each tag


    fig = plot_dist_predictions(df_tag_dist,
                                lang)
    save_graph_to_folder(fig, lang, 'dist_predictions')
    
    
    mistake_freq_df = mistake_frequency_by_word_type(Y_test, 
                                                    predicts_test, 
                                                    list_tags, 
                                                    encoder_tag
                                                    )

    display(mistake_freq_df.head(n=10)) # Print 10 most frequent errors

0it [00:00, ?it/s]

____________________________ KAZAKH CORPUS ____________________________
Size dataset :  (20000, 10)


,Metric,Multi-language POS Tagger,NLTK POS Tagger
0,Test Accuracy,0.857356,0.167992
1,Test F1 Score,0.855359,0.225548
2,Test Recall,0.857356,0.167992
3,Train Accuracy,0.962642,0.200245
4,Train F1 Score,0.961985,0.276446
5,Train Recall,0.962642,0.200245


,Tag,Accuracy
0,ADJ,0.704301
1,NOUN,0.914831
2,VERB,0.798595
3,AUX,0.565217
4,ADV,0.661017
5,PRON,0.886957
6,ADP,0.989011
7,PUNCT,1.000000
8,PROPN,0.878788
9,NUM,0.956522


,Tag,Correct Predictions,Incorrect Predictions
0,ADJ,131,55
1,NOUN,623,58
2,VERB,341,86
3,AUX,39,30
4,ADV,39,20
5,PRON,102,13
6,ADP,90,1
7,PUNCT,141,0
8,PROPN,145,20
9,NUM,66,3


...

,Metric,Multi-language POS Tagger,NLTK POS Tagger
0,Test Accuracy,0.835037,0.285781
1,Test F1 Score,0.834912,0.363433
2,Test Recall,0.835037,0.285781
3,Train Accuracy,0.945036,0.293366
4,Train F1 Score,0.944786,0.377708
5,Train Recall,0.945036,0.293366


,Tag,Accuracy
0,PROPN,0.756477
1,VERB,0.728522
2,ADP,0.945000
3,DET,0.961538
4,NOUN,0.771277
5,CCONJ,0.960526
6,ADJ,0.711268
7,AUX,0.939189
8,PRON,0.926829
9,PUNCT,1.000000


,Tag,Correct Predictions,Incorrect Predictions
0,PROPN,146,47
1,VERB,212,79
2,ADP,189,11
3,DET,175,7
4,NOUN,290,86
5,CCONJ,73,3
6,ADJ,101,41
7,AUX,139,9
8,PRON,228,18
9,PUNCT,99,0


,From Tag,To Tag,Frequency
6,VERB,NOUN,40
22,NOUN,VERB,30
21,NOUN,PROPN,28
1,PROPN,NOUN,28
34,ADJ,NOUN,23
25,NOUN,ADJ,17
8,VERB,AUX,15
2,PROPN,ADJ,13
47,ADV,ADJ,13
7,VERB,ADJ,12


2it [00:25, 12.94s/it]
